In [2]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

--2021-11-13 18:20:43--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.230.167.118, 54.230.167.133, 54.230.167.10, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.230.167.118|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: ‘bm_preproc.py.1’

bm_preproc.py.1     100%[===================>]   9.18K  --.-KB/s    in 0s      

2021-11-13 18:20:44 (42.9 MB/s) - ‘bm_preproc.py.1’ saved [9400/9400]



In [3]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2021-11-13 22:08:56--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.225.125.137, 13.225.125.83, 13.225.125.156, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.225.125.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K   909KB/s    in 0.9s    

2021-11-13 22:08:57 (909 KB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [7]:
from bm_preproc import BoyerMoore

#### Third
Implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return:  
- (a) the number of character comparisons performed.  
- (b) the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.  

In [24]:
def naive(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i+j] != p[j]:
                num_alignments += 1
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, num_alignments, num_character_comparisons

In [18]:
def boyer_moore_2mm(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    alignments = 0
    comparisons = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                alignments += shift
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
            alignments += shift
        i += shift
    return occurrences, alignments, comparisons

In [9]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:1000]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAAGAAAGCAATTTTTGCTGCTAACCTAACATTTCACAATGTCTGGAGACATTTACAGTTCCCACAACCTATGGCAGTTACTGGCATCTACTAGAGGTCAGAGATGCTGGTAAATACTCTGTAATGAACAAGAAGCCCCCCATAGCAAATAAATACCCAGCCCAAGATGGCAATAGTGCCCAGATTGAGAAACTTCACCTTAACCTGATATCATGCAAATATATCTGAAGAAAGACACAAACATAACTAAAGAAAGATGATTACCAGAAGAGATATTCATAAATCTTAGAAGCATAGAAAAAGAAACACAAGGCAATGTTTTCAGTGTCCAGGCAATTATCTTCCTGGGAAAAGCTAGCCTACCAGACCAACATGACTTTTGCACCTTGCTGGCAACCATTCTACTCTTCTGAAGAAGGAGACATCATTTGGACTCTAAAATCCCTTTTTCTGATTTCATACTCATCAAGAAATCTATCCATTTGGCTTAGTTTGTAGCTTATGCTGAAAAACGTGACTTGAGATTTCCTTCACTTGGAAATTGAGATTGCTTAATGTAGATTGACATTCTCAACATTTGGACAATAGTGGGATCAATTATCTTAACTTGCAAAGCTGAAGATTATACCTCTGGGCAACAGTCAAATTACCAAGGTAAATGCTTAGTTGTAGTCAGCATGGGATGGTGTTGAACCACTAATTCCATTTTTTAAAGAGATATAGGGCTTTTCAGGTTCTCTTTTTCTTCTTGAGTGAGCTTAAGTAGTTTGTTTCTTTCAAGGAATTAAACTATTTCATATAAGGTGTCACATTTATTGGCATAAGCTTGTTCAAAATATTTCTTATTATCCTAATATCTGTAGATTTTGTAATGATATCACCTCTCACATTCCTATTTTAAT

In [25]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
occurrences, num_alignments, num_character_comparisons = naive(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799953 984143


In [26]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = boyer_moore_2mm(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799957 165191


In [20]:
print(len(p))

47


In [21]:
t[56922:56922+47]

'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'